# Python for deep learning

In [7]:
import os
if  not os.path.isdir('./training'):
  !cd /content/ && git clone https://github.com/MontpellierRessourcesImagerie/DL_EXP_PC.git && cp -r DL_EXP_PC/data . && cp -r DL_EXP_PC/training .
  %cd DL_EXP_PC

## Cell 7

```python
seed = np.random.randint(0, 2**(32 - 1) - 1) 
rstate = np.random.RandomState(seed)
rstate.shuffle(X)
rstate = np.random.RandomState(seed)
rstate.shuffle(y)
print("first 5 of mixed samples\n", X[0:5], y[0:5])
print("5 samples after the middle\n", X[S:S+5], y[S:S+5])
```

We want to shuffle our data, so that the elements will be in a random order. The ``RandomState`` object from numpy has the method ``shuffle`` that does what we want. However the problem we have is that we need to shuffle X and y in exactly the same way so that the labels in y still correspond to the right input vectors.

To accomplish this, we take advantage of the fact that we are using pseudo-random numbers. A pseudo random number generator generates, starting from a seed-value, a series of numbers which is completly deterministic (i.e. not random), but has the same statistical characteristics as a random sequence. By using the same seed for two random number series, we get two times exactly the same series. We use this to shuffle X and y in the same way.

Here is an example:

In [4]:
a = [ 1 ,  2 ,  3 ,  4 ,  5 ,  6 ,  7,   8 ,  9 , 10 ]
b = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"]

In [6]:
import numpy as np
seed = np.random.randint(0, 2**(32 - 1) - 1) 
rstate = np.random.RandomState(seed)
rstate.shuffle(a)
rstate = np.random.RandomState(seed)
rstate.shuffle(b)
print("a", a)
print("b", b)

a [1, 2, 4, 7, 8, 10, 6, 3, 5, 9]
b ['i', 'ii', 'iv', 'vii', 'viii', 'x', 'vi', 'iii', 'v', 'ix']
